In [2]:
!pip install pandas numpy torch transformers sentence-transformers scikit-learn tqdm

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

ModuleNotFoundError: No module named 'pandas'

In [21]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

In [22]:
data = [
    {"date": "2024-06-01", "review": "Delivery guy was very rude and late"},
    {"date": "2024-06-01", "review": "Food arrived cold"},
    {"date": "2024-06-02", "review": "Delivery partner behaved badly"},
    {"date": "2024-06-02", "review": "The food was stale"},
    {"date": "2024-06-03", "review": "Maps not working properly for delivery"},
    {"date": "2024-06-03", "review": "Bring back 10 minute delivery"},
    {"date": "2024-06-04", "review": "Delivery person was impolite"},
    {"date": "2024-06-04", "review": "Instamart should be open all night"},
    {"date": "2024-06-05", "review": "App keeps crashing while placing order"},
    {"date": "2024-06-06", "review": "Delivery was delayed due to wrong location"},
]

df = pd.DataFrame(data)
df["date"] = pd.to_datetime(df["date"])
df

,date,review
0,2024-06-01,Delivery guy was very rude and late
1,2024-06-01,Food arrived cold
2,2024-06-02,Delivery partner behaved badly
3,2024-06-02,The food was stale
4,2024-06-03,Maps not working properly for delivery
5,2024-06-03,Bring back 10 minute delivery
6,2024-06-04,Delivery person was impolite
7,2024-06-04,Instamart should be open all night
8,2024-06-05,App keeps crashing while placing order
9,2024-06-06,Delivery was delayed due to wrong location


In [23]:
def extract_candidate_topic(review):
    review = review.lower()
    return review[:80]

In [24]:
df["candidate_topic"] = df["review"].apply(extract_candidate_topic)
df

,date,review,candidate_topic
0,2024-06-01,Delivery guy was very rude and late,delivery guy was very rude and late
1,2024-06-01,Food arrived cold,food arrived cold
2,2024-06-02,Delivery partner behaved badly,delivery partner behaved badly
3,2024-06-02,The food was stale,the food was stale
4,2024-06-03,Maps not working properly for delivery,maps not working properly for delivery
5,2024-06-03,Bring back 10 minute delivery,bring back 10 minute delivery
6,2024-06-04,Delivery person was impolite,delivery person was impolite
7,2024-06-04,Instamart should be open all night,instamart should be open all night
8,2024-06-05,App keeps crashing while placing order,app keeps crashing while placing order
9,2024-06-06,Delivery was delayed due to wrong location,delivery was delayed due to wrong location


In [25]:
topic_embeddings = {}
canonical_topics = []

SIM_THRESHOLD = 0.72

def assign_canonical_topic(candidate):
    emb = embedder.encode(candidate)

    best_topic = None
    best_sim = 0

    for topic, topic_emb in topic_embeddings.items():
        sim = cosine_similarity([emb], [topic_emb])[0][0]
        if sim > best_sim:
            best_sim = sim
            best_topic = topic

    if best_sim < SIM_THRESHOLD:
        topic_embeddings[candidate] = emb
        canonical_topics.append(candidate)
        return candidate
    else:
        return best_topic

In [26]:
df["canonical_topic"] = df["candidate_topic"].apply(assign_canonical_topic)
df

,date,review,candidate_topic,canonical_topic
0,2024-06-01,Delivery guy was very rude and late,delivery guy was very rude and late,delivery guy was very rude and late
1,2024-06-01,Food arrived cold,food arrived cold,food arrived cold
2,2024-06-02,Delivery partner behaved badly,delivery partner behaved badly,delivery partner behaved badly
3,2024-06-02,The food was stale,the food was stale,the food was stale
4,2024-06-03,Maps not working properly for delivery,maps not working properly for delivery,maps not working properly for delivery
5,2024-06-03,Bring back 10 minute delivery,bring back 10 minute delivery,bring back 10 minute delivery
6,2024-06-04,Delivery person was impolite,delivery person was impolite,delivery person was impolite
7,2024-06-04,Instamart should be open all night,instamart should be open all night,instamart should be open all night
8,2024-06-05,App keeps crashing while placing order,app keeps crashing while placing order,app keeps crashing while placing order
9,2024-06-06,Delivery was delayed due to wrong location,delivery was delayed due to wrong location,delivery was delayed due to wrong location


In [27]:
trend_report = (
    df.groupby(["canonical_topic", "date"])
      .size()
      .unstack(fill_value=0)
      .sort_index(axis=1)
)

trend_report

date,2024-06-01,2024-06-02,2024-06-03,2024-06-04,2024-06-05,2024-06-06
canonical_topic,,,,,,
app keeps crashing while placing order,0,0,0,0,1,0
bring back 10 minute delivery,0,0,1,0,0,0
delivery guy was very rude and late,1,0,0,0,0,0
delivery partner behaved badly,0,1,0,0,0,0
delivery person was impolite,0,0,0,1,0,0
delivery was delayed due to wrong location,0,0,0,0,0,1
food arrived cold,1,0,0,0,0,0
instamart should be open all night,0,0,0,1,0,0
maps not working properly for delivery,0,0,1,0,0,0


In [28]:
trend_report.to_csv("trend_analysis_report.csv")